# Load images stored

In [1]:
import numpy as np
import pandas as pd
import cv2
from matplotlib import pyplot as plt

In [2]:
def load_images(train, directory):
    
    # This function loads the images, resizes them and puts them into an array
    
    img_size = 224
    train_image = []
    for name in train['image_id']:
        path = directory + 'images/' + name + '.jpg'
        img = cv2.imread(path)
        image = cv2.resize(img, (img_size, img_size))
        train_image.append(image)
    train_image_array = np.array(train_image)
    
    return train_image_array

In [3]:
def load_images_augmented(directory):
    
    # This function loads the augmented images and the augmented csv file
    df_train = pd.read_csv(directory + 'augmented.csv')
    
    train_image = []
    for name in df_train['image_id']:
        path = directory + 'images_resized_augmented/' + name + '.jpg'
        img = cv2.imread(path)
        train_image.append(img)
    train_image_array = np.array(train_image)
    
    return  train_image_array, df_train

In [4]:
directory = 'C:/Users/julen/OneDrive/Escritorio/IA/CS577-Deep-Learning/Project/'
x_train, df_train = load_images_augmented(directory)
df_test = pd.read_csv(directory + 'test.csv')
x_test = load_images(df_test, directory)


# Split training set into train and validation

In [5]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

y_train = df_train[['healthy', 'multiple_diseases', 'rust', 'scab']].to_numpy()

x_train_original, y_train_original = shuffle(x_train, y_train)

x_train, x_val, y_train, y_val = train_test_split(x_train_original, y_train_original, test_size = 0.2)

print('Size of x_train: ', x_train.shape)
print('Size of x_val: ', x_val.shape)

Size of x_train:  (7284, 224, 224, 3)
Size of x_val:  (1821, 224, 224, 3)


In [9]:
#See distribution of samples
y_train.sum(axis = 0)

array([2076,  361, 2511, 2336], dtype=int64)

# First approach: Batch Confusion Norm

In [10]:
import keras
from keras import models
from keras.layers import Input, Dense, Activation, Flatten, Conv2D, Dropout, Multiply
from keras.layers import AveragePooling2D, MaxPooling2D
from keras.models import Model, Sequential
import matplotlib.pyplot as plt
import time
import numpy as np
from keras.applications.resnet import ResNet50

In [11]:
def model_plot(history):

    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'])
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'validation'], loc='upper left')
    plt.show()

In [12]:
def print_score(model, x_test, y_test):
    score = model.evaluate(x_test, y_test, verbose = 0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

In [13]:
input_shape = (224, 224, 3)
model_resnet = ResNet50(include_top = False, weights = 'imagenet', input_shape = input_shape)

In [14]:
def ASSP(output, act_name): # TODO: Check if it is well built. Not like in paper. Maybe different image size?
    ASSP = Conv2D(filters = 2048, kernel_size = (1,1))(output)
    ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 2)(ASSP)
#     ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 4)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 6)(ASSP)
    # ASSP = Conv2D(filters = 2048, kernel_size = (3,3), dilation_rate = 7)(ASSP)

    ASSP = Activation(act_name)(ASSP)
    
    return ASSP

In [15]:
from keras import losses
import keras.backend as K
import tensorflow as tf

def compute_bcn_loss(y_pred):

    
    matrix = K.dot(K.transpose(y_pred), y_pred)
    eigenvalues, eigenvectors = tf.linalg.eigh(matrix)
    bcn_loss = K.sum(eigenvalues) 
    
    print(eigenvalues)
    print(type(eigenvalues))
    print(type(y_pred))
    print('y_pred shape: ', y_pred.shape)
    print('yPred.transpose shape: ', K.transpose(y_pred).shape)
    print('matrix shape: ', matrix.shape)
    print('eigenvalues shape: ', eigenvalues.shape)
    
    print()
    
    
    
    return bcn_loss



def custom_loss(y_true, y_pred):
    
    categorical_crossentropy = losses.categorical_crossentropy(y_true, y_pred)
    
    BCN_loss = compute_bcn_loss(y_pred)
    
    loss_total = categorical_crossentropy + BCN_loss
    
    return loss_total
    
    
    
    
    

In [16]:
inputs = Input(shape = (224, 224, 3))
out = model_resnet(inputs)
features = ASSP(out, 'relu')
attention = ASSP(out, 'sigmoid')
tensor_total = Multiply()([features, attention])
tensor_total_flat = Flatten()(tensor_total)
tensor_total_flat = Dense(256, activation = 'relu')(tensor_total_flat) # TODO: Check if it should be relu or another
tensor_total_flat = Dense(4, activation = 'softmax')(tensor_total_flat)

model = Model(inputs=inputs, outputs= tensor_total_flat)

model.compile(optimizer = 'adam', loss = custom_loss, metrics = ['accuracy'])

print(model.summary())

Tensor("loss/dense_2_loss/custom_loss/SelfAdjointEigV2:0", shape=(4,), dtype=float32)
<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>
y_pred shape:  (None, 4)
yPred.transpose shape:  (4, None)
matrix shape:  (4, 4)
eigenvalues shape:  (4,)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
resnet50 (Model)                (None, 7, 7, 2048)   23587712    input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 7, 7, 2048)   4196352     resnet50[1][0]                   

In [17]:
model.fit(x_train, y_train, batch_size = 2, epochs = 3, validation_data = (x_val, y_val))

Train on 7284 samples, validate on 1821 samples
Epoch 1/3
  58/7284 [..............................] - ETA: 6:19:31 - loss: 8.2397 - accuracy: 0.3793

KeyboardInterrupt: 

In [ ]:
# TODO: In order to set the batch size and each batch with different labels, we should use fit_generator or train_on_batch

In [8]:
tf.test.is_gpu_available()

RuntimeError: cudaGetDevice() failed. Status: cudaGetErrorString symbol not found.

In [9]:
tf.config.list_physical_devices('GPU')

[]

# Simple model

In [19]:
def build_model(x_train, y_train, batch_size, epochs, verbose = True):
    
    
    ##model building
    model = Sequential()
    #convolutional layer with rectified linear unit activation
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu'))
    #32 convolution filters used each of size 3x3
    #again
    model.add(Conv2D(64, (3, 3), activation='relu'))
    #64 convolution filters used each of size 3x3
    #choose the best features via pooling
    model.add(MaxPooling2D(pool_size=(2, 2)))
    #randomly turn neurons on and off to improve convergence
    model.add(Dropout(0.25))
    #flatten since too many dimensions, we only want a classification output
    model.add(Flatten())
    #fully connected to get all relevant data
    model.add(Dense(128, activation='relu'))
    #one more dropout for convergence' sake :) 
    model.add(Dropout(0.5))
    #output a softmax to squash the matrix into output probabilities
    model.add(Dense(4, activation='softmax'))
    
#     model = Sequential()
#     model.add(Conv2D(filters = 10, kernel_size = (5,5), 
#                          padding = 'Valid', activation = 'relu')) 
#     model.add(AveragePooling2D(pool_size = (2,2)))
    
#     model.add(Conv2D(filters = 20, kernel_size = (5,5), 
#                          padding = 'Valid', activation = 'relu')) 
#     model.add(AveragePooling2D(pool_size = (2,2)))
    
#     model.add(Conv2D(filters = 40, kernel_size = (3,3), 
#                          padding = 'Valid', activation = 'relu')) 
#     model.add(AveragePooling2D(pool_size = (2,2)))

#     model.add(Flatten())
#     model_leNet.add(Dense(120, activation = 'relu'))
#     model.add(Dense(84, activation = 'relu'))
#     model.add(Dense(10, activation = 'softmax'))
    
    start_time = time.time()
    model.compile(loss = 'categorical_crossentropy', optimizer = 'adam' , metrics = ['acc'])
    history = model.fit(x = x_train, y = y_train, validation_data = (x_val, y_val), batch_size = batch_size, epochs = epochs, verbose = verbose)
    print("--- %s seconds ---" % (time.time() - start_time))
    
    return (model, history)

In [21]:
model_old, history = build_model(x_train, y_train, batch_size = 32, epochs = 10)
model_plot(history)
print_score(model_old, x_test, y_test)

Train on 7284 samples, validate on 1821 samples
Epoch 1/10
 128/7284 [..............................] - ETA: 13:43 - loss: 5268.3353 - acc: 0.2734

KeyboardInterrupt: 

# First approach: Batch Confusion Norm